# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secretz import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 66609237, 'calculationPrice': 'close', 'change': -2.22, 'changePercent': -0.01424, 'close': 158.97, 'closeSource': 'iifafloc', 'closeTime': 1718130017308, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 159.238, 'delayedPriceTime': 1720894086368, 'extendedChange': -0.17, 'extendedChangePercent': -0.00107, 'extendedPrice': 161.29, 'extendedPriceTime': 1742311587064, 'high': 162.65, 'highSource': 'teeypln1 iria dece umd5', 'highTime': 1692114981525, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 159.368, 'iexCloseTime': 1726280444202, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 165.94, 'iexOpenTime': 1701751016881, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1734600062565, 'latestPrice': 157.66, 'latestSource': 'Close', 'latestTime': 'September 2, 2022', 'latestUpdate': 1709891935715, 'latestVolume': 80659272, 'low': 158.97, 'lowSource': 'mireet5 iu

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data ["latestPrice"]
print(price)
market_cap = data ["marketCap"]
print(f"{market_cap/1000000000000} Trillion")

157.66
2.507719353396 Trillion


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number Of Shares To Buy"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares To Buy


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        "N/A"
    ],
    index = my_columns
    ),
    ignore_index = True
)

C:\Users\Dell\AppData\Local\Temp\ipykernel_10136\887391374.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe.append(


,Ticker,Stock Price,Market Capitalization,Number Of Shares To Buy
0,AAPL,157.66,2507719353396,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [10]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks["Ticker"][:5]:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series([
            stock,
            data["latestPrice"],
            data["marketCap"],
            "N/A"
        ],
        index = my_columns),
    ignore_index = True
    )

C:\Users\Dell\AppData\Local\Temp\ipykernel_10136\1024784122.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Dell\AppData\Local\Temp\ipykernel_10136\1024784122.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Dell\AppData\Local\Temp\ipykernel_10136\1024784122.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Dell\AppData\Local\Temp\ipykernel_10136\1024784122.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Dell\AppData\Local\

In [11]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares To Buy
0,A,132.27,39446126018,N/A
1,AAL,13.10,8574350039,N/A
2,AAP,175.64,10558627858,N/A
3,AAPL,155.86,2513659346906,N/A
4,ABBV,139.92,250208695881,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [12]:
def chunks(lst, n):
    """Yields Successive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
    


In [30]:
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

C:\Users\Dell\AppData\Local\Temp\ipykernel_10136\413399271.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Dell\AppData\Local\Temp\ipykernel_10136\413399271.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Dell\AppData\Local\Temp\ipykernel_10136\413399271.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Dell\AppData\Local\Temp\ipykernel_10136\413399271.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Dell\AppData\Local\

,Ticker,Stock Price,Market Capitalization,Number Of Shares To Buy
0,A,131.80,38013486433,N/A
1,AAL,13.07,8753053876,N/A
2,AAP,178.38,10586236242,N/A
3,AAPL,160.17,2617800679176,N/A
4,ABBV,137.48,247544205763,N/A
...,...,...,...,...
496,YUM,117.45,32311444610,N/A
497,ZBH,110.43,22257032257,N/A
498,ZBRA,301.50,15658307478,N/A
499,ZION,57.31,8576316334,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [53]:
portfolio_size = input("Enter The Value Of Your Portfolio")
try:
    val = float(portfolio_size)
except ValueError:
    print("Enter an integer for the portfolio if you don't have one then Enter 0\n")
    portfolio_size = input("Enter The Value Of Your Portfolio")
    val = float(portfolio_size)

In [65]:
position_size = val/len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i , "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])
final_dataframe

#number_of_apple_shares = position_size/500
#print(math.floor(number_of_apple_shares))

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,131.80,38013486433,151.0
1,AAL,13.07,8753053876,1527.0
2,AAP,178.38,10586236242,111.0
3,AAPL,160.17,2617800679176,124.0
4,ABBV,137.48,247544205763,145.0
...,...,...,...,...
496,YUM,117.45,32311444610,169.0
497,ZBH,110.43,22257032257,180.0
498,ZBRA,301.50,15658307478,66.0
499,ZION,57.31,8576316334,348.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [73]:
writer = pd.ExcelWriter("recommended trades.xlsx", engine = "xlsxwriter")
final_dataframe.to_excel(writer, "Recommended Trades", index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [74]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        "font_color" : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
dollar_format = writer.book.add_format(
    {
        "num_format" : "$0.00",
        "font_color" : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
integer_format = writer.book.add_format(
    {
        "num_format" : "0",
        "font_color" : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [75]:
column_formats = {
    "A" : ["Ticker", string_format],
    "B" : ["Stock Price", dollar_format],
    "C" : ["Market Capitalisation", dollar_format],
    "D" : ["Number of Shares to Buy", integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Tradess'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [76]:
writer.save()